In [ ]:
# ───────────────────────────────────────────────
# 0) 라이브러리 & 데이터 로드
# ───────────────────────────────────────────────
import numpy as np, pandas as pd, optuna, joblib
from pathlib import Path
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, precision_score

# (가상의) PropertyRecommender 인스턴스
from my_reco import reco   # ← 기존 클래스 그대로 사용
# ---------------------------
LOG_F = Path("param_precision_log.csv")
MAX_K = 10
PREC_MIN = 0.60          # Precision 제약
TRIALS_FIRST = 50        # 1차 탐색 trial 수
TRIALS_NEXT  = 20        # 매 주기 추가 trial 수

# ───────────────────────────────────────────────
# 1) 내부 로지스틱 회귀 모델 학습  (분류-회귀)
# ───────────────────────────────────────────────
#   – 기존 prepare_logistic_dataset() 함수 사용
#   – 임시 가중치 (1,1,1,  k=5, 1,1) 으로 후보 생성
# ------------------------------------------------
X_all, y_all = reco.prepare_logistic_dataset(
    w_fin=[1,1,1], k=5, w_prop=[1,1], top_n=5)

X_tr, X_te, y_tr, y_te = train_test_split(
    X_all, y_all, test_size=0.2, stratify=y_all, random_state=42)

scaler = StandardScaler().fit(X_tr)
clf    = LogisticRegression(solver='liblinear', max_iter=200).fit(
            scaler.transform(X_tr), y_tr)

print("▪ 내부 로지스틱 AUC:",
      roc_auc_score(y_te, clf.predict_proba(scaler.transform(X_te))[:,1]))
# 모델을 pickle-like 파일로 저장 (재시동 대비)
joblib.dump(clf,   "logistic_model.joblib")
joblib.dump(scaler,"scaler.joblib")

# ───────────────────────────────────────────────
# 2) Precision 평가 함수  (내부 모델 사용)  ※ 핵심
# ───────────────────────────────────────────────
def precision_at_k(w_fin, k, w_prop, lam_k, company_ids):
    """
    • 가중치·k 로 후보 생성  →  로지스틱 P(y=1|x) 예측
    • 상위 k 매물 Precision@k  →  Penalty 적용 후 반환
    """
    # ------ TODO: 실제 추천 로직 호출 ----------------
    # precs = reco.batch_evaluate(
    #           w_fin=w_fin, k=k, w_prop=w_prop, top_n=k,
    #           company_ids=company_ids,
    #           clf_fixed=clf, scaler=scaler)
    # -------------------------------------------------
    # ↓ 데모용 난수 (0.55~0.70)
    rng = np.random.default_rng(hash(tuple(w_fin)+tuple(w_prop)+tuple([k])) & 0xffff)
    precs = 0.55 + 0.15*rng.random()
    # -------------------------------------------------
    if precs < PREC_MIN:
        return None               # 제약 위반
    return precs - lam_k*(k/MAX_K)

# ───────────────────────────────────────────────
# 3) Optuna objective         (하이퍼파라미터 탐색)
# ───────────────────────────────────────────────
def objective(trial):
    w_fin  = np.array([trial.suggest_float('w1',0,5),
                       trial.suggest_float('w2',0,5),
                       trial.suggest_float('w3',0,5)])
    k      = trial.suggest_int  ('k',1,MAX_K)
    w_prop = np.array([trial.suggest_float('wa',0,5),
                       trial.suggest_float('wp',0,5)])
    lam    = trial.suggest_float('lam',0,1)
    score  = precision_at_k(w_fin,k,w_prop,lam,trainval_ids)
    if score is None:
        raise optuna.exceptions.TrialPruned()
    return score

# ───────────────────────────────────────────────
# 4) 80:20 기업 분할 & 1차 탐색 실행
# ───────────────────────────────────────────────
all_companies = np.array(reco.df_fin.index)
trainval_ids, test_ids = train_test_split(
    all_companies, test_size=0.2, random_state=42)

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=TRIALS_FIRST)

print("◎ 1차 Best:", study.best_value, study.best_params)

# trial 결과 로그 → CSV (누적)
df_trials = study.trials_dataframe()[['params_w1','params_w2','params_w3',
                                      'params_k','params_wa','params_wp',
                                      'params_lam','value']]
df_trials.columns = ['w1','w2','w3','k','wa','wp','lam','precision']
df_trials.to_csv(LOG_F, index=False)

# ───────────────────────────────────────────────
# 5) 로그 기반 서로게이트 회귀  (GradientBoostingRegressor)
# ───────────────────────────────────────────────
def fit_surrogate():
    log_df = pd.read_csv(LOG_F)
    X = log_df.drop(columns=['precision']).values
    y = log_df['precision'].values
    model = make_pipeline(
        StandardScaler(),
        GradientBoostingRegressor(n_estimators=300, random_state=0))
    model.fit(X, y)
    return model

surrogate = fit_surrogate()

# ───────────────────────────────────────────────
# 6) 서로게이트로 ‘유망 후보’ 200개 샘플 & 실제 평가
# ───────────────────────────────────────────────
rng  = np.random.default_rng(0)
cand = rng.uniform(size=(200,7))
cand[:,0:3] *= 5            # w_fin 0~5
cand[:,3]   = (cand[:,3]*MAX_K).astype(int)+1
cand[:,4:6] *= 5            # w_prop 0~5
# lam 그대로 0~1
preds = surrogate.predict(cand)
best_i = preds.argmax()
x_best = cand[best_i]

true_score = precision_at_k(
    w_fin=x_best[:3], k=int(x_best[3]),
    w_prop=x_best[4:6], lam_k=x_best[6],
    company_ids=trainval_ids)

print(\"● Surrogate top-1 pred=%.4f  →  실제=%.4f\" % (preds[best_i], true_score))

# 새 trial을 study.enqueue_trial 로 추가해도 되고,
# 다음 루프에서 objective() 가 그대로 평가하도록 할 수도 있습니다.